In [2]:
from fish.ephys.ephys import load, chop_trials, estimate_onset
from fish.util.fileio import load_image
from glob import glob
import numpy as np

In [3]:
def trigger_data(triggers, window, fnames, average=False):
    from fish.util.fileio import load_image
    from numpy import array
    
    ims_trial = []
    
    for ind, trig in enumerate(triggers):
        
        tr = []        
        for t_w in window:        
            tr.append(load_image(fnames[trig + t_w]))
        tr = array(tr)
        
        
        if (len(ims_trial) == 0) or not average:
            ims_trial.append(tr)
        if average:
            if ind == 0:
                ims_trial[0] = ims_trial[0].astype('float32') / len(triggers) 
            
            ims_trial[0] += tr.astype('float32') / len(triggers)
        
    if average:         
        return ims_trial[0]
    else:
        return array(ims_trial)

In [4]:
dirs = {}
dirs['base'] = '/groups/ahrens/ahrenslab/FROM_TIER2/Yu/SPIM/active_datasets/20170228/fish4/20170228_4_1_gfaprgeco_hucgc_6dpf_shorttrials_20170228_185002/'
dirs['ephys'] = dirs['base'] + 'ephys/'
dirs['raw'] = dirs['base'] + 'raw/'
dirs['proj'] = dirs['base'] + 'projections/'
fnames = glob(dirs['proj'] + 't*')
fnames.sort()
print(len(fnames))

7200


In [5]:
from h5py import File 
epdat = {}
with File(dirs['ephys'] + 'data.mat') as f: 
    for k in f['data'].keys():
        epdat[k] = np.squeeze(f['data'][k][:])

In [6]:
# columns 1 and 2 are the start and stop of stim period
# columns 3 and 4 are start and stop of swim
# the code in the last column of this array: 1 means fish gave up, 3 means fish did not give up
from scipy.io import loadmat
trial_info = loadmat(dirs['ephys'] + 'trial_info.mat')['trial_info']
gu_inds = np.where(trial_info[:,-1] == 1)[0]
ngu_inds = np.where(trial_info[:,-1] == 3)[0]

In [15]:
window = np.arange(-10, 20)

inds_gu = epdat['frame'][trial_info[gu_inds][:,3]].astype('int')[:40]
inds_ngu = epdat['frame'][trial_info[ngu_inds][:,3]].astype('int')[:40]

trig_gu = trigger_data(inds_gu, window, fnames, average=True)
trig_ngu = trigger_data(inds_ngu, window, fnames, average=True)

In [16]:
from skimage.io import imsave
imsave('/groups/ahrens/ahrenslab/davis/tmp/triggered_average_gu.tif', trig_gu.astype('float32'))
imsave('/groups/ahrens/ahrenslab/davis/tmp/triggered_average_ngu.tif', trig_ngu.astype('float32'))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(1,(16,16))
plt.imshow(trig_ave.max(1)[11] - trig_ave.max(1)[9], cmap='gray')
plt.show()